In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import pandas as pd

SALARY_LEMMA_NAME = "hh_vacancies_salary_with_lemma.parquet"
DRIVE_DIR = Path("/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets")

salary_path = DRIVE_DIR / SALARY_LEMMA_NAME

if salary_path.exists():
    df_salary = pd.read_parquet(salary_path)
else:
    from google.colab import files
    uploaded = files.upload()
    df_salary = pd.read_parquet(SALARY_LEMMA_NAME)

print("Размер датасета:", df_salary.shape)

Размер датасета: (14131, 31)


In [ ]:
from scipy.stats import skew

skew_value = skew(df_salary["salary_mid"])
print("Skewness:", skew_value)

Skewness: 1.8119828070765458


In [ ]:
import numpy as np

skew_log = skew(np.log1p(df_salary["salary_mid"]))
print("Skewness (log1p):", skew_log)

Skewness (log1p): 0.2159082304218331


In [ ]:
cat_features = [
    "experience",
    "schedule",
    "employment",
    "professional_role_main"
]

num_features = [
    "text_len",
    "key_skills_count",
    "salary_has_from",
    "salary_has_to",
    "pub_dow"
]

X_meta = df_salary[cat_features + num_features]
y = df_salary["salary_mid"]

print("X shape:", X_meta.shape)
print("y shape:", y.shape)

X shape: (14131, 9)
y shape: (14131,)


# Вариант 1: только метаданные

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ("num", StandardScaler(), num_features)
    ]
)

model_meta = Pipeline([
    ("preprocessor", preprocessor),
    ("model", Ridge(alpha=1.0))
])

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mae_list = []
rmse_list = []
mape_list = []

for train_idx, test_idx in kf.split(X_meta):
    X_train, X_test = X_meta.iloc[train_idx], X_meta.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model_meta.fit(X_train, y_train)
    preds = model_meta.predict(X_test)

    mae_list.append(mean_absolute_error(y_test, preds))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, preds)))
    mape_list.append(mape(y_test, preds))

print("MAE:", np.mean(mae_list))
print("RMSE:", np.mean(rmse_list))
print("MAPE:", np.mean(mape_list))

MAE: 36459.580254193635
RMSE: 53531.396692288065
MAPE: 32.06977418587549


В качестве базовой модели использована Ridge-регрессия на метаданных вакансий.
Модель показала среднюю абсолютную ошибку ~36 тыс. рублей и относительную ошибку ~32%.
Это демонстрирует, что структурные характеристики вакансии уже содержат значимую информацию о заработной плате.

In [ ]:
model_meta.fit(X_meta, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['experience', 'schedule',
                                                   'employment',
                                                   'professional_role_main']),
                                                 ('num', StandardScaler(),
                                                  ['text_len',
                                                   'key_skills_count',
                                                   'salary_has_from',
                                                   'salary_has_to',
                                                   'pub_dow'])])),
                ('model', Ridge())])

In [ ]:
feature_names = model_meta.named_steps["preprocessor"].get_feature_names_out()
coefs = model_meta.named_steps["model"].coef_

import pandas as pd

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values("coef", ascending=False)

coef_df.head(10)

,feature,coef
19,cat__professional_role_main_Агент по недвижимости,158394.796151
32,cat__professional_role_main_Брокер,133091.384776
72,cat__professional_role_main_Коммерческий дирек...,117114.863031
134,cat__professional_role_main_Руководитель групп...,108872.023974
174,cat__professional_role_main_Финансовый директо...,107357.863315
116,cat__professional_role_main_Операционный дирек...,89214.956939
47,cat__professional_role_main_Дата-сайентист,82886.600256
13,cat__professional_role_main_DevOps-инженер,79451.114414
53,cat__professional_role_main_Директор по маркет...,77579.487571
173,cat__professional_role_main_Финансовый аналити...,74061.525920


In [ ]:
coef_df.tail(10)

,feature,coef
159,cat__professional_role_main_Специалист техниче...,-42763.306410
112,cat__professional_role_main_Оператор call-цент...,-44103.641961
119,cat__professional_role_main_Охранник,-44898.086939
131,cat__professional_role_main_Психолог,-45150.220040
21,cat__professional_role_main_Администратор,-45723.087502
70,cat__professional_role_main_Кассир-операционист,-45985.406636
113,"cat__professional_role_main_Оператор ПК, опера...",-50245.868367
49,"cat__professional_role_main_Делопроизводитель,...",-54033.201715
76,"cat__professional_role_main_Копирайтер, редакт...",-55723.953441
103,cat__professional_role_main_Методист,-59830.010882


Интерпретация коэффициентов Ridge показала, что наибольшее влияние на прогноз зарплаты оказывает профессиональная роль. Руководящие и технические позиции (директор, data scientist, DevOps) существенно повышают прогноз, тогда как административные и операционные роли его понижают.

# Вариант 1: метаданные + логарифмирование

In [ ]:
import numpy as np

y_log = np.log1p(y)

In [ ]:
mae_list = []
rmse_list = []
mape_list = []

for train_idx, test_idx in kf.split(X_meta):
    X_train, X_test = X_meta.iloc[train_idx], X_meta.iloc[test_idx]
    y_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

    model_meta.fit(X_train, y_train)
    preds_log = model_meta.predict(X_test)

    preds = np.expm1(preds_log)

    mae_list.append(mean_absolute_error(y_test, preds))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, preds)))
    mape_list.append(mape(y_test, preds))

print("MAE (log model):", np.mean(mae_list))
print("RMSE (log model):", np.mean(rmse_list))
print("MAPE (log model):", np.mean(mape_list))

MAE (log model): 35489.125301533975
RMSE (log model): 54539.09966509008
MAPE (log model): 28.70992212886797


Несмотря на выраженную правостороннюю асимметрию распределения, логарифмирование целевой переменной не привело к существенному улучшению абсолютных метрик (MAE, RMSE). Это объясняется предварительной очисткой данных и отсутствием экстремальных выбросов.

#Вариант 2: только текст

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.9,
    ngram_range=(1,2),
    max_features=50000
)

X_text = df_salary["description_lemma"]

model_text = Pipeline([
    ("tfidf", tfidf),
    ("model", Ridge(alpha=1.0))
])

In [ ]:
mae_list = []
rmse_list = []
mape_list = []

for train_idx, test_idx in kf.split(X_text):
    X_train, X_test = X_text.iloc[train_idx], X_text.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model_text.fit(X_train, y_train)
    preds = model_text.predict(X_test)

    mae_list.append(mean_absolute_error(y_test, preds))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, preds)))
    mape_list.append(mape(y_test, preds))

print("MAE (text):", np.mean(mae_list))
print("RMSE (text):", np.mean(rmse_list))
print("MAPE (text):", np.mean(mape_list))

MAE (text): 33358.75313873873
RMSE (text): 48160.70914649342
MAPE (text): 29.465253923494494


Зарплата сильнее определяется содержанием вакансии (навыками, требованиями, обязанностями), чем формальными характеристиками (тип занятости, день публикации и т.д.).

#Финальная модель — мета + текст

In [ ]:
X_full = df_salary[cat_features + num_features + ["description_lemma"]]

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor_full = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ("num", StandardScaler(), num_features),
        ("text", tfidf, "description_lemma")
    ]
)

model_full = Pipeline([
    ("preprocessor", preprocessor_full),
    ("model", Ridge(alpha=1.0))
])

In [ ]:
mae_list = []
rmse_list = []
mape_list = []

for train_idx, test_idx in kf.split(X_full):
    X_train, X_test = X_full.iloc[train_idx], X_full.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model_full.fit(X_train, y_train)
    preds = model_full.predict(X_test)

    mae_list.append(mean_absolute_error(y_test, preds))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, preds)))
    mape_list.append(mape(y_test, preds))

print("MAE (meta + text):", np.mean(mae_list))
print("RMSE (meta + text):", np.mean(rmse_list))
print("MAPE (meta + text):", np.mean(mape_list))

MAE (meta + text): 31296.67487072135
RMSE (meta + text): 45530.107146612456
MAPE (meta + text): 27.118039838640374


Наиболее информативной является модель, объединяющая текстовые и структурные признаки. Это подтверждает, что уровень заработной платы определяется как формальными характеристиками вакансии, так и содержанием описания.

In [ ]:
y_log = np.log1p(y)

In [ ]:
mae_list = []
rmse_list = []
mape_list = []

for train_idx, test_idx in kf.split(X_full):
    X_train, X_test = X_full.iloc[train_idx], X_full.iloc[test_idx]
    y_train_log = y_log.iloc[train_idx]
    y_test = y.iloc[test_idx]

    model_full.fit(X_train, y_train_log)
    preds_log = model_full.predict(X_test)

    preds = np.expm1(preds_log)

    mae_list.append(mean_absolute_error(y_test, preds))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, preds)))
    mape_list.append(mape(y_test, preds))

print("MAE (meta + text, log):", np.mean(mae_list))
print("RMSE (meta + text, log):", np.mean(rmse_list))
print("MAPE (meta + text, log):", np.mean(mape_list))

MAE (meta + text, log): 29876.972361268003
RMSE (meta + text, log): 46083.53816778114
MAPE (meta + text, log): 23.793351209096624


Логарифмирование целевой переменной позволило снизить относительную ошибку модели (MAPE) и улучшить среднюю абсолютную ошибку. Несмотря на небольшое увеличение RMSE, log-модель демонстрирует более стабильные предсказания и была выбрана как финальная.

In [ ]:
model_full.fit(X_full, y_log)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['experience', 'schedule',
                                                   'employment',
                                                   'professional_role_main']),
                                                 ('num', StandardScaler(),
                                                  ['text_len',
                                                   'key_skills_count',
                                                   'salary_has_from',
                                                   'salary_has_to',
                                                   'pub_dow']),
                                                 ('text',
                                                  TfidfVectorizer(max_df=0.9,
                                                                  max_features=50000,
                                                                  min_df=3,
                                                                  ngram_range=(1,
                                                                               2)),
                                                  'description_lemma')])),
                ('model', Ridge())])

In [ ]:
feature_names = model_full.named_steps["preprocessor"].get_feature_names_out()
coefs = model_full.named_steps["model"].coef_

import pandas as pd

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values("coef", ascending=False)

coef_df.head(20)

,feature,coef
13162,text__зарабатывать,1.003388
11279,text__доход,0.863880
25882,text__опыт,0.782997
7271,text__высокий,0.691730
39638,text__сделка,0.676592
1988,text__vip,0.626966
22889,text__недвижимость,0.570282
11689,text__ежедневный выплата,0.554452
43372,text__стратегия,0.530655
39327,text__свободный,0.524765


In [ ]:
coef_df.tail(20)

,feature,coef
26849,text__отель,-0.397889
3872,text__блогер,-0.402475
26607,text__ответственность,-0.417355
6427,text__воспитатель,-0.429156
45780,text__удалённый работа,-0.434967
6979,text__выдача,-0.439683
24977,text__обязанность доставка,-0.441162
30726,text__поручение,-0.441243
48526,text__час день,-0.445059
30272,text__получить,-0.447368


In [ ]:
text_coef = coef_df[coef_df["feature"].str.contains("text__")]
text_coef.head(20)

,feature,coef
13162,text__зарабатывать,1.003388
11279,text__доход,0.863880
25882,text__опыт,0.782997
7271,text__высокий,0.691730
39638,text__сделка,0.676592
1988,text__vip,0.626966
22889,text__недвижимость,0.570282
11689,text__ежедневный выплата,0.554452
43372,text__стратегия,0.530655
39327,text__свободный,0.524765


In [ ]:
text_coef.tail(20)

,feature,coef
26849,text__отель,-0.397889
3872,text__блогер,-0.402475
26607,text__ответственность,-0.417355
6427,text__воспитатель,-0.429156
45780,text__удалённый работа,-0.434967
6979,text__выдача,-0.439683
24977,text__обязанность доставка,-0.441162
30726,text__поручение,-0.441243
48526,text__час день,-0.445059
30272,text__получить,-0.447368


In [ ]:
meta_coef = coef_df[~coef_df["feature"].str.contains("text__")]
meta_coef.head(15)

,feature,coef
47,cat__professional_role_main_Дата-сайентист,0.351360
19,cat__professional_role_main_Агент по недвижимости,0.296750
53,cat__professional_role_main_Директор по маркет...,0.282309
44,"cat__professional_role_main_Главный врач, заве...",0.277503
138,cat__professional_role_main_Руководитель отдел...,0.274011
32,cat__professional_role_main_Брокер,0.266831
39,cat__professional_role_main_Врач,0.250987
41,cat__professional_role_main_Генеральный директ...,0.246826
72,cat__professional_role_main_Коммерческий дирек...,0.246174
173,cat__professional_role_main_Финансовый аналити...,0.237952


Интерпретация объединённой модели показала, что ключевыми факторами повышения зарплаты являются профессиональная роль и упоминание стратегических, технических или коммерческих задач в описании вакансии. В то время как административные и вспомогательные функции ассоциируются с более низким уровнем оплаты.